In [0]:
WITH parsed AS (
    SELECT
        -- Line 1
        TRY_CAST(SUBSTRING(TLE_LINE1, 3, 5) AS INT) AS NORAD_CAT_ID,
        SUBSTRING(TLE_LINE1, 10, 8) AS INT_DESIGNATOR,
        TRY_CAST(SUBSTRING(TLE_LINE1, 19, 2) AS INT) AS EPOCH_YEAR,
        TRY_CAST(SUBSTRING(TLE_LINE1, 21, 12) AS DOUBLE) AS EPOCH_DAY,

        -- Line 2 orbital elements
        TRY_CAST(SUBSTRING(TLE_LINE2, 9, 8) AS DOUBLE) AS INCLINATION,
        TRY_CAST(SUBSTRING(TLE_LINE2, 18, 8) AS DOUBLE) AS RAAN,
        TRY_CAST(SUBSTRING(TLE_LINE2, 27, 7) AS DOUBLE) / 1e7 AS ECCENTRICITY,
        TRY_CAST(SUBSTRING(TLE_LINE2, 35, 8) AS DOUBLE) AS ARG_OF_PERIGEE,
        TRY_CAST(SUBSTRING(TLE_LINE2, 44, 8) AS DOUBLE) AS MEAN_ANOMALY,
        TRY_CAST(SUBSTRING(TLE_LINE2, 53, 11) AS DOUBLE) AS MEAN_MOTION
    FROM workspace.default.gp_latest_20250813_190840s
)
SELECT
    *,
    -- Mean motion to semi-major axis (n in rev/day, convert to seconds)
    POWER(398600.4418 / POWER((MEAN_MOTION * 2 * PI()) / 86400, 2), 1.0/3.0) AS SEMI_MAJOR_AXIS_KM,
    -- Approx altitude above Earth's surface
    POWER(398600.4418 / POWER((MEAN_MOTION * 2 * PI()) / 86400, 2), 1.0/3.0) - 6371 AS ALTITUDE_KM,
    -- Orbit classification
    CASE
        WHEN POWER(398600.4418 / POWER((MEAN_MOTION * 2 * PI()) / 86400, 2), 1.0/3.0) - 6371 < 2000 THEN 'LEO'
        WHEN POWER(398600.4418 / POWER((MEAN_MOTION * 2 * PI()) / 86400, 2), 1.0/3.0) - 6371 BETWEEN 2000 AND 35786 THEN 'MEO'
        WHEN ABS(POWER(398600.4418 / POWER((MEAN_MOTION * 2 * PI()) / 86400, 2), 1.0/3.0) - 6371 - 35786) < 200 THEN 'GEO'
        ELSE 'HEO'
    END AS ORBIT_CLASS
FROM parsed;

-- Create a new table with parsed and calculated orbital elements
drop table if exists workspace.default.parsed_orbital_elements_0904;
CREATE TABLE workspace.default.parsed_orbital_elements_0904 AS
WITH parsed AS (
    SELECT *,
        -- Line 1
        TRY_CAST(SUBSTRING(TLE_LINE1, 3, 5) AS INT) AS NORAD_CAT_ID_p,
        SUBSTRING(TLE_LINE1, 10, 8) AS INT_DESIGNATOR,
        TRY_CAST(SUBSTRING(TLE_LINE1, 19, 2) AS INT) AS EPOCH_YEAR,
        TRY_CAST(SUBSTRING(TLE_LINE1, 21, 12) AS DOUBLE) AS EPOCH_DAY,

        -- Line 2 orbital elements
        TRY_CAST(SUBSTRING(TLE_LINE2, 9, 8) AS DOUBLE) AS INCLINATION_p,
        TRY_CAST(SUBSTRING(TLE_LINE2, 18, 8) AS DOUBLE) AS RAAN,
        TRY_CAST(SUBSTRING(TLE_LINE2, 27, 7) AS DOUBLE) / 1e7 AS ECCENTRICITY_p,
        TRY_CAST(SUBSTRING(TLE_LINE2, 35, 8) AS DOUBLE) AS ARG_OF_PERIGEE,
        TRY_CAST(SUBSTRING(TLE_LINE2, 44, 8) AS DOUBLE) AS MEAN_ANOMALY_p,
        TRY_CAST(SUBSTRING(TLE_LINE2, 53, 11) AS DOUBLE) AS MEAN_MOTION_p
    FROM workspace.default.gp_latest_20250904_231748s
),
orbit_calc AS (
    SELECT
        *,
        -- Semi-major axis (from mean motion)
        POWER(398600.4418 / POWER((MEAN_MOTION_p * 2 * PI()) / 86400, 2), 1.0/3.0) AS SEMI_MAJOR_AXIS_KM
    FROM parsed
)
SELECT
    *,
    SEMI_MAJOR_AXIS_KM - 6371 AS ALTITUDE_KM,
    -- Perigee and Apogee
    (SEMI_MAJOR_AXIS_KM * (1 - ECCENTRICITY)) - 6371 AS PERIGEE_KM,
    (SEMI_MAJOR_AXIS_KM * (1 + ECCENTRICITY)) - 6371 AS APOGEE_KM,
    -- Orbit classification
    CASE
        WHEN (SEMI_MAJOR_AXIS_KM - 6371) < 2000 THEN 'LEO'
        WHEN (SEMI_MAJOR_AXIS_KM - 6371) BETWEEN 2000 AND 35786 THEN 'MEO'
        WHEN ABS((SEMI_MAJOR_AXIS_KM - 6371) - 35786) < 200 THEN 'GEO'
        ELSE 'HEO'
    END AS ORBIT_CLASS
FROM orbit_calc;




select * from parsed_orbital_elements 
